# Resolução de Sistemas Não Lineares

---


## Iteração Ponto-Fixo

Semelhante ao método de iteração de ponto fixo para encontrar raízes de uma única equação, o método de iteração de ponto fixo pode ser estendido para sistemas não lineares. O método de iteração de ponto fixo procede reorganizando o sistema não linear de modo que as equações tenham a forma:

\begin{align*}
x_1 &= f_1(x_1,x_2,....,x_n) \\
x_2 &= f_2(x_1,x_2,....,x_n) \\
x_3 &= f_3(x_1,x_2,...,x_n)  \\
    ...&                     \\
x_n &=  f_n(x_1,x_2,...,x_n)
\end{align*}

onde $∀ i \le n: f_i$ é uma função não linear dos componentes $x_1,x_2,...,x_n$.

### Exemplo:

Encontre a solução do sistema não linear abaixo:
\begin{align*}
x_1^2 + x_1x_2 &= 10\\
x_2   + 3x_1x_2^2 &= 57
\end{align*}

**Solução**

A solução exata no campo dos números reais para este sistema pode ser obtida usando a biblioteca sympy do python. Veja o código abaixo:

In [ ]:
import sympy as sp

# indica que as impressões do pacote deve seguir formato latex.
sp.init_printing(use_latex=True)
x1, x2 = sp.symbols("x1 x2")
a = sp.nonlinsolve([x1**2 + x1 * x2 - 10, x2 + 3 * x1 * x2**2 - 57], [x1, x2])
display(a)

⎧        ⎛                                                                    
⎪        ⎜                                                                    
⎪        ⎜      3 ___                                  2/3        2/3 3 ______
⎪        ⎜  517⋅╲╱ 2 ⋅√581717⋅(8378775 + 11191⋅√581717)      173⋅2   ⋅╲╱ 83787
⎨(2, 3), ⎜- ────────────────────────────────────────────── - ─────────────────
⎪        ⎜                    911833068                                     26
⎪        ⎜                                                                    
⎪        ⎜                                                                    
⎩        ⎝                                                                    

                                                                              
                                                                              
___________________        2/3         3 _________________________          3 
75 + 11191⋅√581717    2   2   ⋅√581717⋅╲╱ 8378775 +

O método numérico de iteração de ponto fixo requer reorganizar as equações primeiro para a forma:

\begin{align*}
x_1 &= f_1(x_1,x_2) \\
x_2 &= f_2(x_1,x_2) \\
\end{align*}

Uma reorganização possível para esse sistema é dada abaixo:

\begin{align*}
x_1 &= (10 -x_1^2)/x_2 \\
x_2 &= 57 - 3x_1x_2^2  \\
\end{align*}

**Análise de Convergência**

Como sabemos, uma solução real para o sistema acima é $\mathbf{x} = (2,3)$. Para a análise de convergência devemos calcular a matriz Jacobiana que é dada por

$$\mathbf{G} =
\begin{pmatrix}
\frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} \\
\frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2}
\end{pmatrix}
$$

onde
$\frac{\partial f_1}{\partial x_1} = \frac{-2x_1}{x_2} $,
$\frac{\partial f_1}{\partial x_2} = \frac{x_1^2-10}{x_2^2} $,
$\frac{\partial f_2}{\partial x_1} = -3x_2^2 $ e
$
\frac{\partial f_2}{\partial x_1} = -6x_1x_2
$

Calculando os autovalores e o raio espectral com seguinte código python abaixo:

In [ ]:
import numpy as np


# Vamos definir as duas funções de ponto-fixo:
def f1(x1, x2):
    return (10.0 - x1 * x1) / x2


def f2(x1, x2):
    return 57.0 - 3 * x1 * x2 * x2


## E suas derivadas parciais em relação a x1 e x2
def df1dx1(x1, x2):
    return -2.0 * x1 / x2


def df1dx2(x1, x2):
    return (x1 * x1 - 10) / (x2 * x2)


def df2dx1(x1, x2):
    return -3 * x1 * x2


def df2dx2(x1, x2):
    return -6 * x1 * x2


# Alocamos uma matriz 2x2 e atribuidos em dcada posição os
# valores da derivadas
G = np.zeros([2, 2])
G[0][0] = df1dx1(2, 3)
G[0][1] = df1dx2(2, 3)
G[1][0] = df2dx1(2, 3)
G[1][1] = df2dx2(2, 3)

print(G)

# a função eig da biblioteca linealg retorna os autovalores (v) e autovetores (w) da
# de uma matriz.
v, w = np.linalg.eig(G)

print(v)
print(w)


# o raio espectral é o maior autovalor
print("Raio espectral: ", max(abs(v)))

[[ -1.33333333  -0.66666667]
 [-18.         -36.        ]]
[ -0.99056856 -36.34276478]
[[ 0.88933791  0.01903904]
 [-0.45725057  0.99981874]]
Raio espectral:  36.342764778083804


Sabemos que a iteração de ponto-fixo convirjará se $|\rho(\mathbf{G})| \lt 1$. Como podemos observar acima, o $|\rho(\mathbf{G})| \approx  36.3 > 1$, logo a sequencia de aproximações não irá convergir.

Vamos verificar na prática executando o seguinte código python:

In [ ]:
# Implementação do metodo de ponto-fixo usando o arranjo das funções original.
def PontoFixo(x0, tol, maxiter):
    print("%-5s  %-13s %-14s %-8s" % ("Iter", "x1", "x2", "error"))

    # Enquanto i < maxiters
    for i in range(maxiter):
        x0new = f1(x0[0], x0[1])
        x1new = f2(x0new, x0[1])

        x = np.array([x0new, x1new])
        error = np.linalg.norm(x - x0, 2) / np.linalg.norm(x, 2)
        print("%-5d  %6.6e  %6.6e  %6.6e" % (i, x[0], x[1], error))
        if error < tol:
            return x

        x0 = x
    return x


x0 = np.array([2.01, 3.01])
x = PontoFixo(x0, 0.0001, 5)

Iter   x1            x2             error   
0      1.980033e+00  2.367597e+00  2.083642e-01
1      2.567780e+00  2.370268e+01  8.952164e-01
2      1.437182e-01  -4.270867e+03  1.005550e+00
3      -2.336608e-03  -7.864334e+06  9.994569e-01
4      -1.271563e-06  4.335419e+11  1.000018e+00


Um rearranjo diferente para as equações tem a forma:

\begin{align*}
x_1 &= \sqrt{(10 -x_1x_2} \\
x_2 &= \sqrt{ (57 - x_2)/3x_1} \\
\end{align*}

Usamos a mesma aproximação inicial, temos:


In [ ]:
import numpy as np


def f1new(x1, x2):
    return np.sqrt(10.0 - x1 * x2)


def f2new(x1, x2):
    return np.sqrt((57 - x2) / (3 * x1))


def PontoFixoFixed(x0, tol, maxiter):
    print("%-5s  %-10.8s %-10.8s %-8.8s" % ("Iter", "x1", "x2", "error"))
    for i in range(maxiter):
        x1new = f1new(x0[0], x0[1])
        x2new = f2new(x1new, x0[1])

        x = np.array([x1new, x2new])

        error = np.linalg.norm(x - x0, 2) / np.linalg.norm(x, 2)

        print("%-5d  %-8.8f %-8.8f %-8.8f" % (i, x[0], x[1], error))
        if error < tol:
            break
        x0 = x
    return x


x0 = np.array([1.5, 3.5])
x = PontoFixoFixed(x0, 1.0e-8, 40)

Iter   x1         x2         error   
0      2.17944947 2.86050599 0.25945929
1      1.94053388 3.04955067 0.08428602
2      2.02045629 2.98340475 0.02879238
3      1.99302813 3.00570436 0.00980173
4      2.00238524 2.99805430 0.00335241
5      1.99918491 3.00066556 0.00114555
6      2.00027865 2.99977255 0.00039162
7      1.99990475 3.00007776 0.00013386
8      2.00003256 2.99997342 0.00004576
9      1.99998887 3.00000909 0.00001564
10     2.00000380 2.99999689 0.00000535
11     1.99999870 3.00000106 0.00000183
12     2.00000044 2.99999964 0.00000062
13     1.99999985 3.00000012 0.00000021
14     2.00000005 2.99999996 0.00000007
15     1.99999998 3.00000001 0.00000002
16     2.00000001 3.00000000 0.00000001


**Exercício 1**: Faça a análise de convergência para a reorganização acima.

**Exercício 2**: Resolva o sistema abaixo usando o método de ponto-fixo com tolerância de $10^{-6}$.

\begin{align*}
x^3 + y & = 1  \\
y^3 - x & = -1  
\end{align*}


## Método de Newton

O método de Newton-Raphson é o método de escolha para resolver sistemas de equações não lineares. Muitos pacotes de software de engenharia (especialmente software de análise de elementos finitos) que resolvem sistemas de equações não lineares usam o método de Newton-Raphson. A derivação do método para sistemas não lineares é muito semelhante à versão unidimensional para determinação de raízes. Suponha um sistema não linear de equações da forma:


\begin{align*}
f_1(x_1,x_2,....,x_n) &= 0\\
f_2(x_1,x_2,....,x_n) &= 0\\
f_3(x_1,x_2,...,x_n)  &= 0\\
  ...&                     \\
f_n(x_1,x_2,...,x_n) &= 0
\end{align*}


Se os componentes de um iteração $x^{(k)} \in \mathbb{R}^n$ são conhecidos como $x_1^{(k)}, x_2^{(k)},..., x_n^{(k)}$, então aplicando a expansão da serie de taylor para a primeira equação oo redor desses componentes, temos:


\begin{align*}
f_1(x_1^{(k+1)}, x_2^{(k+1)},..., x_n^{(k+1)}) & \approx f_1(x_1^{(k)}, x_2^{(k)},..., x_n^{(k)}) + \\
 & \frac{\partial f_1}{\partial x_1}(x_1^{(k+1)}-x_1^{(k)}) + \\
 & \frac{\partial f_1}{\partial x_2}(x_2^{(k+1)}-x_2^{(k)}) + \\
 & ... \\
 & \frac{\partial f_1}{\partial x_n}(x_n^{(k+1)}-x_n^{(k)})
\end{align*}


Aplicando a mesma expansão para as equações $f_2,..., f_n$ ,chegamos ao seguinte sistema de equações lineares:

$$
\mathbf{F}(\mathbf{x})^{(k+1)} = \mathbf{F}(\mathbf{x})^{(k)} + \mathbf{J}(\mathbf{x}^{(k+1)})(\mathbf{x}^{(k+1)} - \mathbf{x}^{(k)}
$$

Definindo o lado esquerdo para zero (que é o valor desejado para as funções $f_1, f_2, \cdots, f_n$ então, o sistema pode ser escrito como:

$$
\mathbf{0} = \mathbf{F}(\mathbf{x})^{(i)} + \mathbf{J}(\mathbf{x}^{(k+1)})(\mathbf{x}^{(k+1)} - \mathbf{x}^{(k)})  \\
↓ \\
\mathbf{J}(\mathbf{x}^{(k+1)})\mathbf{\Delta x}= - \mathbf{F}(\mathbf{x}^{(k)})
$$

onde $\mathbf{J}$ é a matriz jacobiana dada por:

$$
\mathbf{J}(\mathbf{x})^{(k)}_{ij} = \frac{\partial f_i(\mathbf{x}^{(k)})}{\partial x_j}
$$

A nova aproximação é dada por:  $ \mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \mathbf{\Delta x} $

### Exemplo

Encontre a solução do sistema não linear abaixo:
\begin{align}
x_1^2 + x_1x_2 &= 10\\
x_2   + 3x_1x_2^2 &= 57
\end{align}

**Solução**

Além de exigir uma estimativa inicial, o método de Newton requer a avaliação das derivadas das funções $f_1=x_1^2+x_1x_2-10$ e $f_2=x_2+3x_1x_2^2-57$. Se $\mathbf{J}_{ij}=\frac{\partial f_1}{\partial x_j}$, então tem a seguinte forma:

$$
 \mathbf{J}=\left(\begin{matrix}\frac{\partial f_1}{\partial x_1}& \frac{\partial f_1}{\partial x_2}\\\frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2}\end{matrix}\right)=\left(\begin{matrix}2x_1+x_2& x_1\\3x_2^2 & 1+6x_1x_2\end{matrix}\right)
$$

Assumindo uma estimativa inicial de $\mathbf{x}^{(0)}=(1.5, 3.5)$, então o vetor $\mathbf{F}$ e a matriz $\mathbf{J}$ têm componentes:

$$
\mathbf{F}(\mathbf{x}^{(0)}) =\left(\begin{array}{c}x_1^2+x_1x_2-10\\x_2+3x_1x_2^2-57\end{array}\right)=\left(\begin{array}{c}-2.5\\1.625\end{array}\right) \qquad \mathbf{J}=\left(\begin{matrix}6.5 & 1.5 \\ 36.75 & 32.5\end{matrix}\right)
$$

Os componentes do vetor $\Delta x$ podem ser obtidos resolvendo o sistema abaixo:

$$
\left(\begin{matrix}6.5 & 1.5 \\ 36.75 & 32.5\end{matrix}\right) \Delta \mathbf{x} = -\left(\begin{array}{c}-2.5\\1.625\end{array}\right)
$$

Usando algum método de solução de sistema:


In [ ]:
import numpy as np

J = np.empty([2, 2])
F = np.empty([2, 1])

J[0][0] = 6.5
J[0][1] = 1.5
J[1][0] = 36.75
J[1][1] = 32.5
F[0] = -2.5
F[1] = 1.625
deltaX = np.linalg.solve(J, -F)
print("\Delta x = %0.5f %0.5f" % (x[0], x[1]))

\Delta x = 2.00000 3.00000


Assim, $\mathbf{x}^{(1)} =  \mathbf{x}^{(0)} + \Delta \mathbf{x}$:

$$ x^{(1)}=x^{(0)}+\Delta x=\left(\begin{array}{c}2.036029\\2.843875\end{array}\right)
$$

O erro relativo é dado por:

$$
  \epsilon_r= \frac{\| \Delta \mathbf{x} \|_2}{\| \Delta \mathbf{x}^{(1)}\|_2} = \frac{\sqrt{(0.53603)^2+(-0.65612)^2}}{\sqrt{(2.036029)^2+(2.843875)^2}}=0.2422
$$


Abaixo, o programa completo:

In [ ]:
# Função que calcula a matriz Jacobiana para o exercicio proposto
def CalculaJacobiano(x):
    J = np.empty([2, 2])
    J[0][0] = 2 * x[0] + x[1]
    J[0][1] = x[0]
    J[1][0] = 3 * x[1] ** 2
    J[1][1] = 1 + 6 * x[0] * x[1]
    return J


# Funcão que calcula o vetor Resíduo
def CalculaResiduo(x):
    F = np.empty([2, 1])
    F[0] = x[0] ** 2 + x[0] * x[1] - 10
    F[1] = x[1] + 3 * x[0] * x[1] ** 2 - 57
    return F


# Implementa as iterações do método de Newton
def NewtonSolve(x0, tol, maxiters):
    print("%-5s  %-10.8s %-10.8s %-8.8s" % ("Iter", "x1", "x2", "error"))
    for i in range(maxiters):
        J = CalculaJacobiano(x0)
        F = CalculaResiduo(x0)
        dx = np.linalg.solve(J, -F)
        x = x0 + dx
        # error = np.linalg.norm(x-x0)/np.linalg.norm(x)
        error = np.linalg.norm(dx) / np.linalg.norm(x)
        print("%-5d  %-8.8f %-8.8f %-8.8E" % (i, x[0], x[1], error))
        if error < tol:
            break
        x0 = x
    return x


# Cria o vetor com a aproximação inicial
#
x0 = np.empty([2, 1])
x0[0] = 1.5
x0[1] = 3.5

# Chama o método de Newton para resolver o sistema
x = NewtonSolve(x0, 1.0e-8, 40)

Iter   x1         x2         error   
0      2.03602882 2.84387510 2.42238211E-01
1      1.99870061 3.00228856 4.51244689E-02
2      1.99999998 2.99999941 7.30046491E-04
3      2.00000000 3.00000000 1.62758110E-07
4      2.00000000 3.00000000 2.15280612E-14


## Aproximando J por diferenças finitas



In [ ]:
# Funcão que calcula o vetor Resíduo
def CalculaResiduo(x):
    F = np.empty([2, 1])
    F[0] = x[0] ** 2 + x[0] * x[1] - 10
    F[1] = x[1] + 3 * x[0] * x[1] ** 2 - 57
    return F


def CalculaJacobiano(x):
    eps = 1.0e-8
    J = np.matrix([2, 2])
    u = np.zeros([2, 1])
    for i in range(2):
        for j in range(2):
            u[j] = eps
            J[i][j] = (CalculaResiduo(x + u) - CalculaResiduo(x)) / eps
            u[j] = 0.0
    return J


# Implementa as iterações do método de Newton
def NewtonSolve(x0, tol, maxiters):
    print("%-5s  %-10.8s %-10.8s %-8.8s" % ("Iter", "x1", "x2", "error"))
    for i in range(maxiters):
        J = CalculaJacobiano(x0)
        F = CalculaResiduo(x0)
        dx = np.linalg.solve(J, -F)
        x = x0 + dx
        # error = np.linalg.norm(x-x0)/np.linalg.norm(x)
        error = np.linalg.norm(dx) / np.linalg.norm(x)
        print("%-5d  %-8.8f %-8.8f %-8.8E" % (i, x[0], x[1], error))
        if error < tol:
            break
        x0 = x
    return x


# Cria o vetor com a aproximação inicial
#
x0 = np.empty([2, 1])
x0[0] = 1.5
x0[1] = 3.5

# Chama o método de Newton para resolver o sistema
x = NewtonSolve(x0, 1.0e-6, 40)

NameError: ignored